In [ ]:
#合并电量数据模块

import pandas as pd
import os
import re

# --- 1. 配置 ---
file_path = r"E:\A智网\电量预测数据\lightgbm模型\电量数据源文件"
output_excel_file = r"E:\A智网\电量预测数据\lightgbm模型\湖北省行业日用电量.xlsx"
file_list = [
    "湖北行业客户用电量日时序电量监测2023-01-31.xls", "湖北行业客户用电量日时序电量监测2023-02-28.xls",
    "湖北行业客户用电量日时序电量监测2023-03-31.xls", "湖北行业客户用电量日时序电量监测2023-04-30.xls",
    "湖北行业客户用电量日时序电量监测2023-05-31.xls", "湖北行业客户用电量日时序电量监测2023-06-30.xls",
    "湖北行业客户用电量日时序电量监测2023-07-31.xls", "湖北行业客户用电量日时序电量监测2023-08-31.xls",
    "湖北行业客户用电量日时序电量监测2023-09-30.xls", "湖北行业客户用电量日时序电量监测2023-10-31.xls",
    "湖北行业客户用电量日时序电量监测2023-11-30.xls", "湖北行业客户用电量日时序电量监测2023-12-31.xls",
    "湖北行业客户用电量日时序电量监测2024-01-31.xls", "湖北行业客户用电量日时序电量监测2024-02-29.xls",
    "湖北行业客户用电量日时序电量监测2024-03-31.xls", "湖北行业客户用电量日时序电量监测2024-04-30.xls",
    "湖北行业客户用电量日时序电量监测2024-05-31.xls", "湖北行业客户用电量日时序电量监测2024-06-30.xls",
    "湖北行业客户用电量日时序电量监测2024-07-31.xls", "湖北行业客户用电量日时序电量监测2024-08-31.xls",
    "湖北行业客户用电量日时序电量监测2024-09-30.xls", "湖北行业客户用电量日时序电量监测2024-10-31.xls",
    "湖北行业客户用电量日时序电量监测2024-11-30.xls", "湖北行业客户用电量日时序电量监测2024-12-31.xls",
    "湖北行业客户用电量日时序电量监测2025-01-31.xls", "湖北行业客户用电量日时序电量监测2025-02-28.xls",
    "湖北行业客户用电量日时序电量监测2025-03-31.xls", "湖北行业客户用电量日时序电量监测2025-04-30.xls",
    "湖北行业客户用电量日时序电量监测2025-05-31.xls", "湖北行业客户用电量日时序电量监测2025-06-30.xls",
    "湖北行业客户用电量日时序电量监测2025-07-31.xls", "湖北行业客户用电量日时序电量监测2025-08-31.xls"
]

# 用于存放每个月处理好的【宽格式】DataFrame
all_monthly_wide_dfs = []

print("开始处理Excel (.xls) 文件 (直接宽表合并模式)...")

# --- 2. 循环处理每个文件，生成月度宽表 ---
for file_name in file_list:
    full_file_path = os.path.join(file_path, file_name)
    try:
        year_match = re.search(r'(\d{4})-\d{2}-\d{2}', file_name)
        if not year_match: continue
        year = year_match.group(1)
        
        print(f"正在读取和处理文件: {file_name}")
        
        # 步骤 2.1: 读取文件，得到多级标题的DataFrame
        df_month = pd.read_excel(full_file_path, header=[0, 1], skiprows=1, index_col=0)
        
        # 步骤 2.2: 清理第二级标题中的空格，确保能匹配到'本年'
        df_month.columns = pd.MultiIndex.from_tuples(
            [(str(col[0]).strip(), str(col[1]).strip()) for col in df_month.columns]
        )
        
        # 步骤 2.3: 直接筛选出所有'本年'的列
        df_this_year = df_month.loc[:, (slice(None), '本年')]

        # 步骤 2.4: 格式化列标题为 YYYYMMDD
        # a) 丢掉'本年'这一级标题
        date_columns = df_this_year.columns.get_level_values(0)
        # b) 将'MM/DD'格式的日期转换为'YYYYMMDD'
        formatted_columns = []
        for date_str in date_columns:
            try:
                # 使用pd.to_datetime进行稳健的日期转换和格式化
                full_date = pd.to_datetime(f"{year}-{date_str}")
                formatted_columns.append(full_date.strftime('%Y%m%d'))
            except ValueError:
                # 如果某个列名无法转换为日期（例如'行业名称'），则保持原样
                formatted_columns.append(date_str)
        
        # c) 将清理好的列名赋回DataFrame
        df_this_year.columns = formatted_columns
        
        all_monthly_wide_dfs.append(df_this_year)
        print(f"文件 {file_name} 处理成功！")
        
    except Exception as e:
        print(f"处理文件 {file_name} 时发生错误: {e}")

# --- 3. 横向合并所有月度宽表并保存 ---
if all_monthly_wide_dfs:
    print("\n所有文件处理完毕，开始合并...")
    
    # 步骤 3.1: 使用pd.concat横向拼接所有DataFrame
    # pandas会根据行索引(行业名称)自动对齐数据
    final_wide_df = pd.concat(all_monthly_wide_dfs, axis=1)
    
    # 步骤 3.2: 将行索引'行业名称'转换回普通列
    final_wide_df = final_wide_df.reset_index()
    # 可能需要重命名索引列
    final_wide_df.rename(columns={'index': '行业名称'}, inplace=True)

    # 步骤 3.3: 保存到您指定的Excel文件
    final_wide_df.to_excel(output_excel_file, index=False)
    
    print("\n处理完成！")
    print(f"数据已按您的要求格式化并保存到文件: {output_excel_file}")
    print("\n最终输出格式预览:")
    print(final_wide_df.iloc[:5, :5])
else:
    print("\n没有成功处理任何数据，请检查上面的错误信息。")

开始处理Excel (.xls) 文件 (直接宽表合并模式)...
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-01-31.xls
文件 湖北行业客户用电量日时序电量监测2023-01-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-02-28.xls
文件 湖北行业客户用电量日时序电量监测2023-02-28.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-03-31.xls
文件 湖北行业客户用电量日时序电量监测2023-03-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-04-30.xls
文件 湖北行业客户用电量日时序电量监测2023-04-30.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-05-31.xls
文件 湖北行业客户用电量日时序电量监测2023-05-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-06-30.xls
文件 湖北行业客户用电量日时序电量监测2023-06-30.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-07-31.xls
文件 湖北行业客户用电量日时序电量监测2023-07-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-08-31.xls
文件 湖北行业客户用电量日时序电量监测2023-08-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-09-30.xls
文件 湖北行业客户用电量日时序电量监测2023-09-30.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-10-31.xls
文件 湖北行业客户用电量日时序电量监测2023-10-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-11-30.xls
文件 湖北行业客户用电量日时序电量监测2023-11-30.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-12-31.xls
文件 湖北行业客户用电量日时序电量监测202

In [ ]:
#湖北每日温度预处理

import pandas as pd

# --- 1. 配置您的文件信息 ---
weather_file_path = r"E:\A智网\电量预测数据\lightgbm模型\气象数据\气象数据手动处理.xlsx"
sheet_name = "湖北温度"
output_filename_excel = r"E:\A智网\电量预测数据\lightgbm模型\湖北省每日温度特征.xlsx"

print(f"开始处理文件: {weather_file_path} (采用直接构建模式)")

# --- 2. 加载和处理数据 ---
try:
    df_hubei = pd.read_excel(weather_file_path, sheet_name=sheet_name)
    
    if df_hubei.empty:
        print("错误：读取到的工作表为空，处理中止。")
    else:
        # a) 找出所有15分钟温度数据列
        temp_cols = df_hubei.filter(regex=r'^T\d{4,}$').columns
        
        # b) 对每一天（每一行）计算统计指标
        daily_stats = df_hubei[temp_cols].agg(['mean', 'max', 'min', 'std'], axis=1)
        df_processed = pd.concat([df_hubei[['YMD']], daily_stats], axis=1)

        # c) 使用中文名重命名并添加温差
        df_processed.rename(columns={
            'mean': '日平均温度', 'max': '日最高温度',
            'min': '日最低温度', 'std': '日温度标准差'
        }, inplace=True)
        df_processed['日温差'] = df_processed['日最高温度'] - df_processed['日最低温度']
        
        # d) 将YMD设为索引，然后转置
        df_processed.set_index('YMD', inplace=True)
        df_transposed = df_processed.T

        # --- 3. 【核心修正】手动构建最终的DataFrame ---

        # a) 将转置后的索引（'日平均温度'等）变成一个真正的列
        df_final = df_transposed.reset_index()

        # b) 将这个新列的名字改为'温度特征'
        df_final.rename(columns={'index': '温度特征'}, inplace=True)

        # c) 将所有日期列的数据格式化为保留三位小数
        date_cols = df_final.columns[1:] # 获取所有日期列
        df_final[date_cols] = df_final[date_cols].round(3)

        print("\n数据处理完成，最终DataFrame在内存中的预览：")
        # 预览前5列，确认结构完全正确
        print(df_final.iloc[:, :5])

        # --- 4. 保存到Excel ---
        #    我们直接将这个完美的df_final写入文件
        #    index=False 是必须的，因为我们已经手动处理好了第一列
        #    header=True 也是必须的，它会写入'温度特征', '20230101', ... 这一行
        df_final.to_excel(output_filename_excel, index=False, header=True)
        
        print("\n" + "="*50)
        print("      数据已成功保存！")
        print(f"      文件路径: {output_filename_excel}")
        print("="*50)

except FileNotFoundError:
    print(f"错误：文件未找到！请确认路径是否正确: {weather_file_path}")
except Exception as e:
    print(f"处理过程中发生错误: {e}")

开始处理文件: E:\A智网\电量预测数据\lightgbm模型\气象数据\气象数据手动处理.xlsx (采用直接构建模式)

数据处理完成，最终DataFrame在内存中的预览：
YMD    温度特征  20230101  20230102  20230103  20230104
0     日平均温度     6.883     7.165     5.711     6.257
1     日最高温度     8.776     9.739    11.349    11.328
2     日最低温度     5.495     5.712     1.355     0.901
3    日温度标准差     1.090     1.315     3.217     3.881
4       日温差     3.281     4.027     9.994    10.427

      数据已成功保存！
      文件路径: E:\A智网\电量预测数据\lightgbm模型\湖北省每日温度特征.xlsx


In [ ]:
import pandas as pd
from pathlib import Path # 导入用于处理文件路径的Pathlib库

# --- 1. 配置您的文件路径 ---
# 【请确保这两个路径是正确的】
load_file_path = r"E:\A智网\电量预测数据\lightgbm模型\湖北省行业日用电量.xlsx"
temp_file_path = r"E:\A智网\电量预测数据\lightgbm模型\湖北省每日温度特征.xlsx"

# --- 【核心修改】动态构建输出路径 ---
# 1. 将输入路径字符串转换为一个Path对象，方便我们操作
input_path = Path(load_file_path)
# 2. 使用 .parent 属性获取该文件所在的文件夹路径
output_directory = input_path.parent
# 3. 定义我们想要的输出文件名
output_filename = "电量与天气对齐后的最终数据.csv"
# 4. 使用 / 操作符安全地将文件夹路径和文件名拼接起来，形成完整的输出路径
output_final_csv = output_directory / output_filename

print(f"输入文件路径: {load_file_path}")
print(f"输出文件将保存在同一文件夹下: {output_directory}")
print(f"最终输出文件路径: {output_final_csv}")


# --- 2. 加载并转换【电量数据】(宽表 -> 长表) ---
try:
    print(f"\n正在加载电量数据...")
    df_load_raw = pd.read_excel(load_file_path)
    total_load_row = df_load_raw[df_load_raw['行业名称'].str.contains('全行业用电合计', na=False)]
    if total_load_row.empty:
        raise ValueError("在电量文件中未找到'全行业用电合计'行")
    df_load_processed = total_load_row.set_index('行业名称').T
    df_load_processed.columns = ['total_load_MWh']
    df_load_processed.index.name = 'date'
    df_load_processed.index = pd.to_datetime(df_load_processed.index, format='%Y%m%d')
    print("电量数据已成功转换为长表格式。")
except Exception as e:
    print(f"处理电量文件时出错: {e}")
    df_load_processed = pd.DataFrame()


# --- 3. 加载并转换【温度数据】(宽表 -> 长表) ---
try:
    print(f"\n正在加载温度数据...")
    df_temp_raw = pd.read_excel(temp_file_path)
    df_temp_processed = df_temp_raw.set_index('温度特征').T
    df_temp_processed.index.name = 'date'
    df_temp_processed.index = pd.to_datetime(df_temp_processed.index, format='%Y%m%d')
    df_temp_processed.rename(columns={
        '日平均温度': 'temp_mean', '日最高温度': 'temp_max', '日最低温度': 'temp_min',
        '日温度标准差': 'temp_std', '日温差': 'temp_range'
    }, inplace=True)
    print("温度数据已成功转换为长表格式。")
except Exception as e:
    print(f"处理温度文件时出错: {e}")
    df_temp_processed = pd.DataFrame()


# --- 4. 对齐、填充与合并 ---
if not df_load_processed.empty and not df_temp_processed.empty:
    print("\n开始对齐电量和温度数据...")
    start_date = min(df_load_processed.index.min(), df_temp_processed.index.min())
    end_date = max(df_load_processed.index.max(), df_temp_processed.index.max())
    master_index = pd.date_range(start=start_date, end=end_date, freq='D')
    
    df_load_aligned = df_load_processed.reindex(master_index)
    df_temp_aligned = df_temp_processed.reindex(master_index)
    
    print("正在对缺失的温度数据进行线性插值填充...")
    df_temp_filled = df_temp_aligned.interpolate(method='linear')
    
    df_final = df_load_aligned.join(df_temp_filled)
    df_final.dropna(subset=['total_load_MWh'], inplace=True)
    
    print("\n最终合并完成！")
    print("最终可用于模型训练的数据预览：")
    print(df_final.head())
    
    # 使用我们动态构建的路径来保存文件
    df_final.to_csv(output_final_csv, encoding='utf_8_sig')
    print("\n" + "="*50)
    print("      数据已成功保存！")
    print(f"      文件已保存在: {output_final_csv}")
    print("="*50)

else:
    print("\n因数据加载失败，无法继续进行对齐和合并操作。")